## Preprocessing

Aufgabe 7: Preprocessing & Pipelines
Scikit-Learn ermöglicht es mittels Pipelines, verschiedene Vorverarbeitungsschritte (Normalisierung,
Dimensionalitätsreduktion, etc.) mit einem Klassifier zu verbinden. In dieser Aufgabe beschäftigen
wir uns mit dem Workflow von Datenverarbeitungsschritten mittels Pipelines.\

a) Laden Sie den Breast Cancer Wisconsin dataset unter:
https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancerwisconsin/
wdbc.data\
Dieses Dataset beinhaltet ein binary Label für Krebsdiagnostic (M=malignant, B=benign)
und mehrere numerische Features, berechnet aus digitalisierten Bildern von Zellkernen.

b) Nutzen Sie die sklearn.preprocessing.LabelEncoder um die binary Label in ein numerisches
Attribut zu kodieren.\

c) Mit sklearn.cross_validation.train_test_split splitten Sie die Daten in Trainings (80%) und
Testdaten (20%). Setzen Sie random_state=1.

d) Als Preprocessingschritte nutzen Sie sklearn.preprocessing.StandardScaler für feature
scaling und für Dimensionalitätsreduktion ein PCA mit sklearn.decomposition. PCA und mit
n_components=2. Als Klassifikator nutzen Sie sklearn.linear_model.LogisticRegression mit
random_state=1. Packen Sie alle Schritte in eine Pipeline (sklearn.pipeline.Pipeline).

e) Testen Sie mittels pipeline.score die Genauigkeit des Modells. Sie sollten eine Genauigkeit
von 0. 947 erreichen.

f) Statt PCA nutzen Sie ein Recursive Feature Elimination (RFE) zur Feature-Auswahl
(sklearn.feature_selection.RFECV). Wie viele und welche Features sind für die Klassifikation
interessant? Welche (max.) Genauigkeit auf die Testdaten kann mit diesem Schritt (statt
PCA) erreicht werden?

a) Laden Sie den Breast Cancer Wisconsin dataset unter:
https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancerwisconsin/
wdbc.data\
Dieses Dataset beinhaltet ein binary Label für Krebsdiagnostic (M=malignant, B=benign)
und mehrere numerische Features, berechnet aus digitalisierten Bildern von Zellkernen.

In [1]:
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data

--2022-05-10 19:07:22--  https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 124103 (121K) [application/x-httpd-php]
Saving to: ‘wdbc.data.1’

wdbc.data.1         100%[===================>] 121.19K   585KB/s    in 0.2s    

2022-05-10 19:07:22 (585 KB/s) - ‘wdbc.data.1’ saved [124103/124103]



In [2]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Exercise suggests to import sklearn.cross_validation.train_test_split
# this is depricated for scikit-learn 1.0.2
# suggestion: instead of 
# from sklearn.cross_validation import train_test_split 
# use:
from sklearn.model_selection import train_test_split


from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA 

from sklearn.feature_selection import RFECV
from sklearn.svm import SVR


In [3]:
filename = 'wdbc.data'

columns = ['id', 'diagnosis']
df_original = pd.read_csv(filename, header=None) 
###0 identifier, 1 diagnosis

b) Nutzen Sie die sklearn.preprocessing.LabelEncoder um die binary Label in ein numerisches
Attribut zu kodieren.

In [4]:

le = LabelEncoder()
le.fit(df_original[1])
df = df_original.copy()
labels = le.transform(df_original[1])
# to transform labels back to original use: le.inverse_transform() 

df = df_original.copy()

# remove labels and original identifier
# although not specified explicitly the unique sample identifier should not contain any 
# important data for classification and is hence removed. Also labels shouldn't be contained in the input dataframe.
df = df.drop([0,1], axis=1)
df

,2,3,4,5,6,7,8,9,10,11,...,22,23,24,25,26,27,28,29,30,31
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


c) Mit sklearn.cross_validation.train_test_split splitten Sie die Daten in Trainings (80%) und
Testdaten (20%). Setzen Sie random_state=1.

In [5]:
# sklearn.cross_validation.train_test_split is depricated for scikit-learn 1.0.2
# instead sklearn.model_selection.train_test_split was used 

# split in train 80% and testdata
X_train, X_test, y_train, y_test  = train_test_split(df, labels, random_state=1, test_size=0.20)

d) Als Preprocessingschritte nutzen Sie sklearn.preprocessing.StandardScaler für feature
scaling und für Dimensionalitätsreduktion ein PCA mit sklearn.decomposition. PCA und mit
n_components=2. Als Klassifikator nutzen Sie sklearn.linear_model.LogisticRegression mit
random_state=1. Packen Sie alle Schritte in eine Pipeline (sklearn.pipeline.Pipeline).

In [6]:
steps = [('scaler', StandardScaler()), 
         ('pca', PCA(n_components=2)), 
         ('classifier', LogisticRegression(random_state=1))]
pipe = Pipeline(steps)

pipe.fit(X=X_train, y=y_train)

Pipeline(steps=[('scaler', StandardScaler()), ('pca', PCA(n_components=2)),
                ('classifier', LogisticRegression(random_state=1))])

e) Testen Sie mittels pipeline.score die Genauigkeit des Modells. Sie sollten eine Genauigkeit
von 0. 947 erreichen.

In [7]:
pipe.score(X_test, y_test)

0.9473684210526315

f) Statt PCA nutzen Sie ein Recursive Feature Elimination (RFE) zur Feature-Auswahl
(sklearn.feature_selection.RFECV). Wie viele und welche Features sind für die Klassifikation
interessant? Welche (max.) Genauigkeit auf die Testdaten kann mit diesem Schritt (statt
PCA) erreicht werden?

In [8]:
estimator = SVR(kernel="linear")
selector = RFECV(estimator)

steps_rfe = [('scaler', StandardScaler()), 
         ('RFE', selector), 
         ('classifier', LogisticRegression(random_state=1))]

pipe_rfe = Pipeline(steps_rfe)

pipe_rfe.fit(X=X_train, y=y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('RFE', RFECV(estimator=SVR(kernel='linear'))),
                ('classifier', LogisticRegression(random_state=1))])

In [9]:
# Wie viele und welche Features sind für die Klassifikation interessant?

print('Wieviele: '+ str(selector.support_.sum()))
print('Welche: ' + str(selector.support_))
print('Welche: (ranking)' + str(selector.ranking_ ))

Wieviele: 25
Welche: [ True  True  True  True  True  True  True  True False  True  True  True
  True  True  True  True  True  True  True False  True False  True  True
 False  True  True  True False  True]
Welche: (ranking)[1 1 1 1 1 1 1 1 3 1 1 1 1 1 1 1 1 1 1 4 1 6 1 1 5 1 1 1 2 1]


Es werden 25 Features selektiert. Davon ist insbesondere Nummer 9 (Symmetrie) sowie einige der Features die mit Feature Engineering erzeugt wurden uninteressant. Für einge genauere Beschreibung der einzelnen Features siehe: https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.names

In [10]:
#Welche (max.) Genauigkeit auf die Testdaten kann mit diesem Schritt (statt PCA) erreicht werden?
pipe_rfe.score(X_test, y_test)

0.9736842105263158

Es wird eine Geauigkeit von 0.9736842105263158 erreicht.